In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")
example = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")

# A first glance

In [ ]:
#Nan
print(train.isna().sum())
print(test.isna().sum())

# Visualization

In [ ]:
#Dataframe

print(train.columns.values.tolist())
train.head()

Let's review each variable with graphs and see which one may be more useful.

## Pclass

In [ ]:
#Data
Variable="Pclass"

print(train[Variable].unique())
print(train[Variable].value_counts())

sns.catplot(x = Variable, y = 'Survived',
            data = train, kind = 'bar',height=6, aspect=2,margin_titles = True)

plt.title('Survival Probability ' + Variable, fontsize = (13))
plt.ylabel('Survival Probability')
plt.show()

sns.catplot(x = Variable, y = 'Survived', hue = 'Sex',
            data = train, kind = 'bar',height=6, aspect=2,margin_titles = True)

plt.title('Survival Probability by Sex and ' + Variable, fontsize = (13))
plt.ylabel('Survival Probability')
plt.show()

No surprises in the first graph: first class people have a higher survival rate than the second class and the second. The same phenomenon happends between the second and third class.
Women in first and second class have a similar survival rate. The same phenomenon happens between men in the second and third class.

## Name
At the begging I directly assume that the name hasn't influence in the survival rate, but after a close examination a discover a important detail.

In [ ]:
train['Name_bin'] = train['Name'].str.extract('( [A-Z]+\w*\.)', expand=False).str.strip()
#train['Name_bin'] = np.where(train['Name_bin'].isin(['Mr.','Miss.','Mrs.','Master.',"Don.","Rev.","Dr."]), train['Name_bin'], 'somebodysomebod')

train['Name_bin'] = np.where(train['Name_bin'] == "Mme.","Mrs.",train['Name_bin'])
train['Name_bin'] = np.where(train['Name_bin'] == "Don.","Mr.",train['Name_bin'])
train['Name_bin'] = np.where(train['Name_bin'] == "Mlle.","Miss.",train['Name_bin'])
train['Name_bin'] = np.where(train['Name_bin'] == "Ms.","Mrs.",train['Name_bin'])

train['Name_bin'] = np.where(train['Name_bin'] == "Major.","Military",train['Name_bin'])
train['Name_bin'] = np.where(train['Name_bin'] == "Col.","Military",train['Name_bin'])
train['Name_bin'] = np.where(train['Name_bin'] == "Capt.","Military",train['Name_bin'])

train['Name_bin'] = np.where(train['Name_bin'] == "Master.","Proffesion",train['Name_bin'])
train['Name_bin'] = np.where(train['Name_bin'] == "Dr.","Proffesion",train['Name_bin'])
train['Name_bin'] = np.where(train['Name_bin'] == "Rev.","Proffesion",train['Name_bin'])

train['Name_bin'] = np.where(train['Name_bin'] == "Sir.","Royal",train['Name_bin'])
train['Name_bin'] = np.where(train['Name_bin'] == "Lady.","Royal",train['Name_bin'])
train['Name_bin'] = np.where(train['Name_bin'] == "Countess.","Royal",train['Name_bin'])
train['Name_bin'] = np.where(train['Name_bin'] == "Jonkheer.","Royal",train['Name_bin'])




pd.set_option('display.max_rows', None)

print(train)#.value_counts())

def plot_bar(dataframe,colname):
    survived = dataframe[dataframe['Survived']==1][colname].value_counts()
    dead = dataframe[dataframe['Survived']==0][colname].value_counts()
    result = pd.DataFrame([survived,dead], index=['Survived','dead'])
    return result

plt.figure(figsize=(7,6))
plot_bar(train,'Name_bin').plot(kind='bar')
plt.show()


After this graph I decide to merge some categories, and do the same transformation for the test data:

In [ ]:
train['Name_bin'] = np.where(train['Name_bin'] == "Royal","Other",train['Name_bin'])
train['Name_bin'] = np.where(train['Name_bin'] == "Proffesion","Other",train['Name_bin'])
train['Name_bin'] = np.where(train['Name_bin'] == "Military","Other",train['Name_bin'])

plt.figure(figsize=(7,6))
plot_bar(train,'Name_bin').plot(kind='bar')
plt.show()

test['Name_bin'] = test['Name'].str.extract('( [A-Z]+\w*\.)', expand=False).str.strip()
test['Name_bin'] = np.where(test['Name_bin'].isin(['Mr.','Miss.','Mrs.']), test['Name_bin'], 'Other')
print(train['Name_bin'].value_counts())
print(test['Name_bin'].value_counts())



## Sex

In [ ]:
#Data
Variable="Sex"

print(train[Variable].unique())
print(train[Variable].value_counts())

sns.catplot(x = Variable, y = 'Survived',
            data = train, kind = 'bar',height=6, aspect=2,margin_titles = True)

plt.title('Survival Probability ' + Variable, fontsize = (13))
plt.ylabel('Survival Probability')
plt.show()

It is one of the most important variables. In fact one model in which all women survived and all men died has an acuraccy of 0.7655.

## Age

A classification is necessary to be able to see the data more clearly. I am going to use bins to classificate them

In [ ]:
Variable="Age"

train["Baby"] = np.where(train["Age"] <5,1,0)
train["Children"] = np.where((train["Age"] >=5) & (train["Age"] <12),1,0)
train["Teeneger"] = np.where((train["Age"] >=12) & (train["Age"] <20),1,0)
train["Young"] = np.where((train["Age"] >=20) & (train["Age"] <30),1,0)
train["Adult"] = np.where((train["Age"] >=30) & (train["Age"] <50),1,0)
train["Senior"] = np.where(train["Age"] >50,1,0)

test["Baby"] = np.where(test["Age"] <5,1,0)
test["Children"] = np.where((test["Age"] >=5) & (test["Age"] <12),1,0)
test["Teeneger"] = np.where((test["Age"] >=12) & (test["Age"] <20),1,0)
test["Young"] = np.where((test["Age"] >=20) & (test["Age"] <30),1,0)
test["Adult"] = np.where((test["Age"] >=30) & (test["Age"] <50),1,0)
test["Senior"] = np.where(test["Age"] >50,1,0)


Variable="Baby"
sns.catplot(x = Variable, y = 'Survived',
            data = train, kind = 'bar',height=2, aspect=2,margin_titles = True)

Variable="Children"
sns.catplot(x = Variable, y = 'Survived',
            data = train, kind = 'bar',height=2, aspect=2,margin_titles = True)

Variable="Teeneger"
sns.catplot(x = Variable, y = 'Survived',
            data = train, kind = 'bar',height=2, aspect=2,margin_titles = True)

Variable="Young"
sns.catplot(x = Variable, y = 'Survived',
            data = train, kind = 'bar',height=2, aspect=2,margin_titles = True)

Variable="Adult"
sns.catplot(x = Variable, y = 'Survived',
            data = train, kind = 'bar',height=2, aspect=2,margin_titles = True)

Variable="Senior"
sns.catplot(x = Variable, y = 'Survived',
            data = train, kind = 'bar',height=2, aspect=2,margin_titles = True)


Conclusion: There are not linear efects in age.

## SibSp

In [ ]:
#Data
Variable="SibSp"

print(train[Variable].unique())
print(train[Variable].value_counts())

sns.catplot(x = Variable, y = 'Survived',
            data = train, kind = 'bar',height=6, aspect=2,margin_titles = True)

plt.title('Survival Probability ' + Variable, fontsize = (13))
plt.ylabel('Survival Probability')
plt.show()

sns.catplot(x = Variable, y = 'Survived', hue = 'Sex',
            data = train, kind = 'bar',height=6, aspect=2,margin_titles = True)

plt.title('Survival Probability by Sex and ' + Variable, fontsize = (13))
plt.ylabel('Survival Probability')
plt.show()

In [ ]:
#Data
Variable="Parch"

print(train[Variable].unique())
print(train[Variable].value_counts())

sns.catplot(x = Variable, y = 'Survived',
            data = train, kind = 'bar',height=6, aspect=2,margin_titles = True)

plt.title('Survival Probability ' + Variable, fontsize = (13))
plt.ylabel('Survival Probability')
plt.show()

sns.catplot(x = Variable, y = 'Survived', hue = 'Sex',
            data = train, kind = 'bar',height=6, aspect=2,margin_titles = True)

plt.title('Survival Probability by Sex and ' + Variable, fontsize = (13))
plt.ylabel('Survival Probability')
plt.show()

There are no linear efect related with this feature.

## Ticket
I directly assume that the number of the ticket hasn't influence in the survival rate.

## Fare

In [ ]:
Variable="Fare"

print(train[Variable].unique())
print(train[Variable].value_counts())

train["Fare"] = np.where(train["Fare"] >300,300,train["Fare"])
test["Fare"] = np.where(test["Fare"] >300,300,test["Fare"])

With this distribution of data I am going to log normalize the distribution but first I have to complete the Nan values in the test set with the most common value in the train set (8.0500). Also I have eliminated the outlayers.

## Cabin

Hypothesis: the letter of the cabin is related to the location inside the ship and therefore to the survival rate.

In [ ]:
#Data
Variable="Letter"

train["Cabin"] = train["Cabin"].fillna("Z")
train[Variable]=train["Cabin"].astype(str).str[0]

print(train[Variable].unique())
print(train[Variable].value_counts())

sns.catplot(x = Variable, y = 'Survived',
            data = train, kind = 'bar',height=6, aspect=2,margin_titles = True)

plt.title('Survival Probability ' + Variable, fontsize = (13))
plt.ylabel('Survival Probability')
plt.show()


sns.catplot(x = Variable, y = 'Survived', hue = 'Pclass',
            data = train, kind = 'bar',height=6, aspect=2,margin_titles = True)

plt.title('Survival Probability by Sex and ' + Variable, fontsize = (13))
plt.ylabel('Survival Probability')
plt.show()

Most of the people has unkonwn cabin ("Z"). That make the this analysis impractical.

## Embarked

Hypothesis: The embarkation site is related to the location inside the ship and therefore to the survival rate.

In [ ]:
#Data
Variable="Embarked"

print(train[Variable].unique())
print(train[Variable].value_counts())

sns.catplot(x = Variable, y = 'Survived',
            data = train, kind = 'bar',height=6, aspect=2,margin_titles = True)

plt.title('Survival Probability ' + Variable, fontsize = (13))
plt.ylabel('Survival Probability')
plt.show()

sns.catplot(x = Variable, y = 'Survived', hue = 'Sex',
            data = train, kind = 'bar',height=6, aspect=2,margin_titles = True)

plt.title('Survival Probability by Sex and ' + Variable, fontsize = (13))
plt.ylabel('Survival Probability')
plt.show()

I can try to include this variable and see the results.

In [ ]:
#One hot encoding of "Sex", "Parch" and "Embarked"
Sex_train=pd.get_dummies(train["Sex"],prefix='Sex')
Parch_train=pd.get_dummies(train["Parch"],prefix='Parch')
Embarked_train=pd.get_dummies(train["Embarked"],prefix='Embarked')
Class_train=pd.get_dummies(train["Pclass"],prefix='Pclass')
Names=pd.get_dummies(train["Name_bin"],prefix='Name')

train = pd.concat([train, Sex_train,Parch_train,Embarked_train,Class_train,Names], axis=1)

Sex_test=pd.get_dummies(test["Sex"],prefix='Sex')
Parch_test=pd.get_dummies(test["Parch"],prefix='Parch')
Embarked_test=pd.get_dummies(test["Embarked"],prefix='Embarked')
Class_test=pd.get_dummies(test["Pclass"],prefix='Pclass')
Names=pd.get_dummies(test["Name_bin"],prefix='Name')

test = pd.concat([test, Sex_test,Parch_test,Embarked_test,Class_test,Names], axis=1)


train.head()

In [ ]:
#Normalize Fare
train["Fare"] = train["Fare"].fillna(8.0500)
test["Fare"] = test["Fare"].fillna(8.0500)

train["Fare"]=train["Fare"]+1
test["Fare"]=test["Fare"]+1

train['log_Fare'] = np.log(train["Fare"])
test['log_Fare'] = np.log(test["Fare"])

Max_Fare=max([test['log_Fare'].abs().max(),train['log_Fare'].abs().max()])
print(Max_Fare)

train["Normalized_Fare"] = train['log_Fare']/Max_Fare

test["Normalized_Fare"] = train['log_Fare']/Max_Fare

train.head()

plt.figure()
_ = plt.hist(test["Normalized_Fare"], bins=100)


As the plot shows 'Fare' have now a much nicer distribution.

In [ ]:
#Normalize Age
#test["Age"] = test["Age"].fillna(25)
#train["Age"] = train["Age"].fillna(25)

train["Age"] = train["Age"].fillna(25)
test["Age"] = test["Age"].fillna(25)

Max_Age=max([test["Age"].abs().max(),train["Age"].abs().max()])

print(Max_Age)

train["Age_Norm"]=train["Age"]/Max_Age
test["Age_Norm"]=test["Age"]/Max_Age

print(train.isna().sum())
print(test.isna().sum())

In [ ]:
trainShow=train.loc[:,['Survived','Sex_female', 'Sex_male', 'Pclass_1', 'Pclass_2', 'Pclass_3',"Parch","SibSp",
                       'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Normalized_Fare',"Age_Norm",'Name_Mr.','Name_Miss.','Name_Mrs.','Name_Other']]
sns.heatmap(trainShow.corr(),annot=True,cmap='RdYlGn',linewidths=0.2) #data.corr()-->correlation matrix
fig=plt.gcf()
fig.set_size_inches(20,12)
plt.show()

In [ ]:
#Selected variables
trainY=train.loc[:,['Survived']]
trainX=train.loc[:,['Sex_female', 'Sex_male', 'Pclass_1', 'Pclass_2', 'Pclass_3',"Parch","SibSp",  
                'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Normalized_Fare',"Age_Norm",
                    'Name_Mr.','Name_Miss.','Name_Mrs.','Name_Other']]
testX=test.loc[:,['Sex_female', 'Sex_male', 'Pclass_1', 'Pclass_2', 'Pclass_3',"Parch","SibSp", 
                    'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Normalized_Fare',"Age_Norm",
                'Name_Mr.','Name_Miss.','Name_Mrs.','Name_Other']]

print(trainX.isna().sum())
print(testX.isna().sum())

print(trainX["Name_Mr."].head())
trainX.head()

In [ ]:
# MODELING

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

import shap
from catboost import Pool
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier

from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# PERFORMING 50-FOLD CROSS VALIDATION

def cross_val(model):
    
    cv_scores = []
    
    for i in range(0,30):
        
        X_train,X_val,y_train,y_val = train_test_split(trainX,trainY,random_state=40, test_size = 0.2, shuffle = True)

        mod = model
        model = mod.fit(X_train,y_train.values.ravel())
        mod_pred = model.predict(X_val)
        mod_score = model.score(X_val,y_val.values.ravel())
        cv_scores.append(mod_score)
    return cv_scores



logistic_reg = cross_val(LogisticRegression())
SGD_Class = cross_val(SGDClassifier())
decision_tree = cross_val(DecisionTreeClassifier())
random_forest = cross_val(RandomForestClassifier(n_estimators=100))
SVClass = cross_val(SVC())
GradBoost = cross_val(GradientBoostingClassifier())
CatBoost = cross_val(CatBoostClassifier())

cv_scores = pd.DataFrame(columns = ['Logistic_Class','SGDClassifier',
                                    'DecisionTreeCl','RandomF_Class','SV_Class',
                                    'GradientBoost','Catboost'])

cv_scores['Logistic_Class'] = logistic_reg
cv_scores['SGDClassifier'] = SGD_Class
cv_scores['DecisionTreeCl'] = decision_tree
cv_scores['RandomF_Class'] = random_forest
cv_scores['SV_Class'] = SVClass
cv_scores['GradientBoost'] = GradBoost
cv_scores['Catboost'] = CatBoost

In [ ]:
cv_scores

In [ ]:
# CV_means

logistic_mean = np.mean(logistic_reg)
SGD_mean = np.mean(SGD_Class)
decision_tree_mean = np.mean(decision_tree)
randomf_mean = np.mean(random_forest)
SVC_mean = np.mean(SVClass)
GradBoost_mean = np.mean(GradBoost)
CatBoost_mean = np.mean(CatBoost)

cv_means = [logistic_mean,SGD_mean,decision_tree_mean,randomf_mean,SVC_mean,GradBoost_mean,CatBoost_mean]

means = pd.DataFrame(cv_scores.columns, columns = ['Algorithms'])
means['cv_means'] = cv_means
means = means.sort_values(by=['cv_means'], ascending=False)
means

In [ ]:
#Lets use all model at he same time:

def metamodel(model):
    
    model_Y = []
    
    for i in range(0,30):
        
        X_train,X_val,y_train,y_val = train_test_split(trainX,trainY,random_state=40, test_size = 0.2, shuffle = True)

        mod = model
        model = mod.fit(trainX,trainY.values.ravel())
        mod_pred = model.predict_proba(testX)
    return mod_pred

def metamodel_2(model):
    
    model_Y = []
    
    for i in range(0,30):
        
        X_train,X_val,y_train,y_val = train_test_split(trainX,trainY,random_state=40, test_size = 0.2, shuffle = True)

        mod = model
        model = mod.fit(trainX,trainY.values.ravel())
        mod_pred = model.predict(testX)
    return mod_pred

logistic_reg = metamodel(LogisticRegression())
SGD_Class = metamodel_2(SGDClassifier())
decision_tree = metamodel(DecisionTreeClassifier())
random_forest = metamodel(RandomForestClassifier(n_estimators=100))
SVClass = metamodel_2(SVC())
GradBoost = metamodel(GradientBoostingClassifier())
CatBoost = metamodel(CatBoostClassifier())

model_Y = pd.DataFrame(columns = ['Logistic_Class',
                                  'SGDClassifier',
                                    'DecisionTreeCl','RandomF_Class',
                                  'SV_Class',
                                    'GradientBoost','Catboost'])





In [ ]:
model_Y['Logistic_Class'] = logistic_reg[:,1]
model_Y['SGDClassifier'] = SGD_Class
model_Y['DecisionTreeCl'] = decision_tree[:,1]
model_Y['RandomF_Class'] = random_forest[:,1]
model_Y['SV_Class'] = SVClass
model_Y['GradientBoost'] = GradBoost[:,1]
model_Y['Catboost'] = CatBoost[:,1]
model_Y['Union'] = (model_Y['Logistic_Class']+model_Y['SGDClassifier'] +model_Y['DecisionTreeCl']+model_Y['RandomF_Class']+
                    model_Y['SV_Class']+model_Y['GradientBoost']+model_Y['Catboost'])/7

model_Y['Union_discrete'] = np.where(model_Y['Union'] > 0.5,1,0)

print(model_Y)

In [ ]:
#Train accuracy
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score

RFc = RandomForestClassifier(n_estimators = 10, max_features = 8, random_state=0).fit(trainX, trainY.values.ravel())

model=LogisticRegression()#LogisticRegression()
LR=model.fit(trainX, trainY.values.ravel())

y_pred_train = LR.predict(trainX)

conf_mx = confusion_matrix(y_pred_train,trainY.values.ravel())

heat_cm = pd.DataFrame(conf_mx, columns=np.unique(y_pred_train), index = np.unique(y_pred_train))
heat_cm.index.name = 'Actual'
heat_cm.columns.name = 'Predicted'
plt.figure(figsize = (6,5))
sns.set(font_scale=1.4) # For label size
sns.heatmap(heat_cm, cmap="Blues", annot=True, annot_kws={"size": 16},fmt='g')# font size
plt.show()

In [ ]:
y_pred_test = LR.predict(testX)#[:, 1]

test["Survived"]=y_pred_test
#test["Survived"]=model_Y['Union_discrete']
Result=test.loc[:,["PassengerId",'Survived']]


#Obtein the file
Result.to_csv('my_submission.csv', index=False)

print(Result)



## Results
In this markdown I'm showing the progres of my model.

### Iteration 0: Baseline
Goal: My Baseline is provided as an example, in which survivers are selected only by gender. 
 + Train accuracy = 0.8922
 + Test accuracy = 0.7655 
 
Conclusion: Whatever model which results are equal or worse than the baseline is almost worthless.




### Iteration 1: Firt simple model
Goal: In this first model the goal is to create a structure to be improved, take into account some of the variables (Age is not included) and obtain a model to which incremental improvements are added.
 + Train accuracy = 0.8698
 + Test accuracy = 0.7727 
 
Conclusion: It is a very small improvement over the Baseline. The model is still clearly overfitted, but I prefer to work including new variables, as Age, and then work in that direction.




### Iteration 2: Data analysis
Goal: In this second model the goal is to do a previous treatment of the data that allows us to discard useless data, enhancing the useful ones and simplifying the model.
 + Train accuracy = 0.8720
 + Test accuracy = 0.7608 
 
Conclusion: It is not an improvement over the Baseline. The model is still clearly overfitted. Let's work in the model.




### Iteration 3: Different models
Goal: Lets try different models and see which one is better. 
 + Train accuracy = 0.8307
 + Test accuracy = 0.7631 
 
Conclusion: RandomTree was the best model but still is not an improvement over the Baseline. Let's keep thinking and improving.



### Iteration 4: All models
Goal: Lets try different mixing the differente model and see if this is uselful. 
 + Train accuracy = 0.8245
 + Test accuracy = 0.7656
 
Conclusion: There is almost an improvement over the Baseline. Let's try with the data again.



### Iteration 5: New dat analysis
Goal: Get a better understanding of the data. 
 + Train accuracy = 0.8193
 + Test accuracy = 0.7751
 
Conclusion: Best result so far with Logistic Model. Let's try with the data again.



### Iteration 6: All models
Goal: Lets try different mixing the differente model and see if this is uselful. 
 + Train accuracy = 0.8245
 + Test accuracy = 0.7775
 
Conclusion: Anothner small improvement.





